<a href="https://colab.research.google.com/github/CasCard/Machile-Learning-Project/blob/master/Titanic_passenger_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import,division,print_function,unicode_literals

try:
    %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

TensorFlow 2.x selected.
2.0.0


In [0]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"
train_file_path=tf.keras.utils.get_file("train.csv",TRAIN_DATA_URL)
test_file_path=tf.keras.utils.get_file("eval.csv",TEST_DATA_URL)


In [0]:
np.set_printoptions(precision=3,suppress=True)

In [10]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [0]:
LABEL_COLUMN='survived'
LABELS=[0,1]

In [15]:
def get_dataset(file_path,**kwargs):
  dataset=tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs
  )
  return dataset

raw_train_data=get_dataset(train_file_path)
raw_test_data=get_dataset(test_file_path)


Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [0]:
def show_batch(dataset):
  for batch,label in dataset.take(1):
    for key,value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [17]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'male' b'female']
age                 : [36. 33.  6. 60. 38.]
n_siblings_spouses  : [1 1 0 0 0]
parch               : [2 0 1 0 0]
fare                : [27.75  53.1   12.475 26.55  13.   ]
class               : [b'Second' b'First' b'Third' b'First' b'Second']
deck                : [b'unknown' b'E' b'E' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'n' b'n' b'y' b'y']


In [18]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [27. 21. 21. 31. 25.]
n_siblings_spouses  : [1 0 0 0 1]
class               : [b'First' b'Third' b'Third' b'Third' b'First']
deck                : [b'E' b'unknown' b'unknown' b'unknown' b'B']
alone               : [b'n' b'y' b'y' b'y' b'n']


In [21]:
SELECT_COLUMNS=['survived','age','n_siblings_spouses','parch','fare']
DEFAULTS=[0,0.0,0.0,0.0,0.0]
temp_dataset=get_dataset(train_file_path,
                         select_columns=SELECT_COLUMNS,
                         column_defaults=DEFAULTS)

show_batch(temp_dataset)

age                 : [35. 47. 45.  5. 29.]
n_siblings_spouses  : [1. 0. 0. 2. 0.]
parch               : [0. 0. 0. 1. 0.]
fare                : [83.475 15.    26.55  19.258  9.5  ]


In [0]:
example_batch,label_batch=next(iter(temp_dataset))

In [0]:
def pack(features,label):
  return tf.stack(list(features.values()),axis=-1),label

In [24]:
packed_dataset=temp_dataset.map(pack)

for features,labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy)

[[ 18.      0.      0.      9.842]
 [ 63.      0.      0.      9.587]
 [ 42.      0.      0.    227.525]
 [  4.      1.      1.     16.7  ]
 [ 28.      1.      0.     15.5  ]]

<bound method _EagerTensorBase.numpy of <tf.Tensor: id=371, shape=(5,), dtype=int32, numpy=array([1, 1, 1, 1, 1], dtype=int32)>>


In [26]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'female' b'female' b'male']
age                 : [25. 16. 19. 24. 50.]
n_siblings_spouses  : [1 0 1 0 1]
parch               : [0 0 0 0 0]
fare                : [  7.775   7.75    7.854  13.    106.425]
class               : [b'Third' b'Third' b'Third' b'Second' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'F' b'C']
embark_town         : [b'Southampton' b'Queenstown' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'n' b'y' b'n' b'y' b'n']


In [0]:
example_batch,labels_batch=next(iter(temp_dataset))

In [0]:
class PackNumericFeatures(object):
  def __init__(self,names):
    self.names=names 

  def __call__(self,features,labels):
    numeric_freatures=[features.pop(name) for name in self.names]
    numeric_features=[tf.cast(feat,tf.float32) for feat in numeric_freatures]
    numeric_features=tf.stack(numeric_features,axis=-1)
    features['numeric']=numeric_features

    return features,labels

In [0]:
NUMERIC_FEATURES=['age','n_siblings_spouses','parch','fare']

packed_train_data=raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data=raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))


In [32]:
show_batch(packed_train_data)

sex                 : [b'female' b'male' b'male' b'male' b'male']
class               : [b'Third' b'Third' b'Third' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'E' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'y' b'y' b'y' b'y' b'y']
numeric             : [[30.     0.     0.     8.663]
 [11.     0.     0.    18.788]
 [28.     0.     0.     7.25 ]
 [35.     0.     0.    26.288]
 [28.     0.     0.     7.75 ]]


In [0]:
example_batch,labels_batch=next(iter(packed_train_data))

In [34]:
import pandas as pd
desc=pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [0]:
MEAN=np.array(desc.T['mean'])
STD=np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data,mean,std):
  return (data-mean)/std


import functools

In [41]:
normalizer=functools.partial(normalize_numeric_data,mean=MEAN,std=STD)

numeric_column=tf.feature_column.numeric_column('numeric',normalizer_fn=normalizer,shape=[len(NUMERIC_FEATURES)])
numeric_columns=[numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f27c231e158>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [42]:
example_batch['numeric']

<tf.Tensor: id=468, shape=(5, 4), dtype=float32, numpy=
array([[28.   ,  0.   ,  0.   ,  7.896],
       [28.   ,  0.   ,  0.   , 56.496],
       [48.   ,  1.   ,  0.   , 76.729],
       [28.   ,  8.   ,  2.   , 69.55 ],
       [22.   ,  0.   ,  0.   ,  7.125]], dtype=float32)>

In [43]:
numeric_layer=tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 , -0.474, -0.479, -0.485],
       [-0.13 , -0.474, -0.479,  0.405],
       [ 1.468,  0.395, -0.479,  0.776],
       [-0.13 ,  6.476,  2.043,  0.644],
       [-0.61 , -0.474, -0.479, -0.499]], dtype=float32)

In [0]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [0]:
categorical_columns=[]
for feature,vocab in CATEGORIES.items():
  cat_col=tf.feature_column.categorical_column_with_vocabulary_list(
      key=feature,vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))
  

In [46]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [47]:
categorical_layer=tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [0]:
preprocessing_layer=tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)


In [49]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.13  -0.474
 -0.479 -0.485  1.     0.   ]


In [0]:
model=tf.keras.Sequential([
                           preprocessing_layer,
                           tf.keras.layers.Dense(128,activation='relu'),
                           tf.keras.layers.Dense(128,activation='relu'),
                           tf.keras.layers.Dense(1,activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [52]:
train_data=packed_train_data.shuffle(500)
test_data=packed_test_data
model.fit(train_data,epochs=20)

Epoch 1/20
126/126 [==============================] - 3s 21ms/step - loss: 0.5076 - accuracy: 0.7847
Epoch 2/20
126/126 [==============================] - 1s 5ms/step - loss: 0.4225 - accuracy: 0.8134
Epoch 3/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3990 - accuracy: 0.8278
Epoch 4/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3923 - accuracy: 0.8309
Epoch 5/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3770 - accuracy: 0.8389
Epoch 6/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3645 - accuracy: 0.8437
Epoch 7/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3624 - accuracy: 0.8421
Epoch 8/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3638 - accuracy: 0.8501
Epoch 9/20
126/126 [==============================] - 1s 4ms/step - loss: 0.3542 - accuracy: 0.8405
Epoch 10/20
126/126 [==============================] - 1s 5ms/step - loss: 0.3404 - accuracy: 0.853

In [53]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features_2 (DenseFeatu multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  3200      
_________________________________________________________________
dense_1 (Dense)              multiple                  16512     
_________________________________________________________________
dense_2 (Dense)              multiple                  129       
Total params: 19,841
Trainable params: 19,841
Non-trainable params: 0
_________________________________________________________________


In [54]:
test_loss,test_accuracy=model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss,test_accuracy))

53/53 [==============================] - 1s 10ms/step - loss: 0.5163 - accuracy: 0.7803


Test Loss 0.5162930737407703, Test Accuracy 0.7803030014038086


In [55]:
predictions=model.predict(test_data)

for prediction,survived in zip(predictions[:10],list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 37.18%  | Actual outcome:  SURVIVED
Predicted survival: 99.44%  | Actual outcome:  DIED
Predicted survival: 71.65%  | Actual outcome:  SURVIVED
Predicted survival: 50.76%  | Actual outcome:  SURVIVED
Predicted survival: 11.03%  | Actual outcome:  SURVIVED
